## Notebook for the Crossflow Paper

### Import Libaries

In [1]:
#!mkdir data
#!gdown 1CVAQDuPOiwm8h9LJ8a_oOs6zOWS6EgkB
#!gdown 1ykZ9fjTxUwdiEwqagoYZiMcD5aG-7rHe
#!unzip -o test.zip -d data
#!unzip -o train.zip -d data
from google.colab import drive
drive.mount('/content/drive')
!git clone https://github.com/Mamiglia/challenge.git

Mounted at /content/drive
Cloning into 'challenge'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 98 (delta 39), reused 72 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (98/98), 21.03 MiB | 15.85 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from pathlib import Path
from tqdm import tqdm

from challenge.src.common import load_data, prepare_train_data, generate_submission

### Create Neural Network Architectures

- VAE-ENCODER (1024) -> LATENT SPACE (1536) -> VAE-DECODER (1024) train a VAE in parallel with the crossflow network
- CROSSFLOW GETS THE LATENT SPACE FROM VAE AS INPUT
-> INPUT (1024) -> VAE-ENCODER (1536) -> INPUT FOR CROSSFLOW -> CROSSFLOW TRANSFORMER -> OUTPUT FOR CROSSFLOW (1536)
- Use Text Embeddings as input for vae and image embeddings for crossflow
![image.png](attachment:image.png)

In [3]:
class MLPBlock(nn.Module):
    """Simple MLP block with LayerNorm, activation, and dropout"""
    def __init__(self, in_dim, out_dim, dropout=0.0, activation=nn.GELU):
        super().__init__()
        self.block = nn.Sequential(
            nn.LayerNorm(in_dim),
            nn.Linear(in_dim, out_dim),
            activation(),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.block(x)


class ContextMapperVAE(nn.Module):
    """
    Configurable VAE:
    - input_dim: dimension of the context embeddings
    - latent_dim: dimension of the target/image latent
    - num_layers: number of hidden layers for encoder/decoder
    - hidden_dim: width of hidden layers
    - dropout: dropout probability
    """
    def __init__(self, input_dim, latent_dim, num_layers=2, hidden_dim=512, dropout=0.1):
        super().__init__()
        # ---------------- Encoder ----------------
        enc_layers = []
        dim_in = input_dim
        for _ in range(num_layers):
            enc_layers.append(MLPBlock(dim_in, hidden_dim, dropout))
            dim_in = hidden_dim
        self.encoder_backbone = nn.Sequential(*enc_layers)
        self.encoder_head = nn.Linear(hidden_dim, latent_dim * 2)  # μ and logσ

        # ---------------- Decoder ----------------
        dec_layers = []
        dim_in = latent_dim
        for _ in range(num_layers):
            dec_layers.append(MLPBlock(dim_in, hidden_dim, dropout))
            dim_in = hidden_dim
        self.decoder_backbone = nn.Sequential(*dec_layers)
        self.decoder_head = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        # --- Encoder path ---
        h = self.encoder_backbone(x)
        stats = self.encoder_head(h)
        mu, log_sigma = stats.chunk(2, dim=-1)
        sigma = torch.exp(log_sigma)
        eps = torch.randn_like(mu)
        z0 = mu + sigma * eps

        # --- Decoder path ---
        h_dec = self.decoder_backbone(z0)
        x_recon = self.decoder_head(h_dec)
        return z0, mu, log_sigma, x_recon

    def kl_loss(self, mu, log_sigma):
        # KL(q(z|x) || N(0,1))
        return -0.5 * torch.sum(1 + 2 * log_sigma - mu.pow(2) - torch.exp(2 * log_sigma), dim=-1).mean()


In [4]:
class TransformerFlow(nn.Module):
    """
    Configurable Transformer Flow model:
    - latent_dim: dimension of z_t
    - num_layers: number of transformer encoder layers
    - num_heads: number of attention heads
    - ff_dim: feed-forward hidden dimension
    - dropout: dropout in transformer layers
    """
    def __init__(self, latent_dim, num_layers=4, num_heads=4, ff_dim=512, dropout=0.1):
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=latent_dim,
            nhead=num_heads,
            dim_feedforward=ff_dim,
            dropout=dropout,
            activation='gelu',
            batch_first=True,
            norm_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output = nn.Linear(latent_dim, latent_dim)

    def forward(self, z_t, t):
        # Add sinusoidal time embedding
        t_embed = self.time_embedding(t, z_t.size(-1))
        x = z_t + t_embed
        x = x.unsqueeze(1)  # transformer expects sequence
        x = self.transformer(x)
        return self.output(x.squeeze(1))

    def time_embedding(self, t, dim):
        half_dim = dim // 2
        freqs = torch.exp(
            torch.arange(half_dim, device=t.device) * (-torch.log(torch.tensor(10000.0)) / (half_dim - 1))
        )
        angles = t.unsqueeze(1) * freqs.unsqueeze(0)
        emb = torch.cat([torch.sin(angles), torch.cos(angles)], dim=-1)
        return emb


In [5]:
@torch.no_grad()
def integrate_flow(flow, z0, n_steps=20):
    z = z0.clone()
    t_values = torch.linspace(0, 1, n_steps, device=z0.device)
    dt = 1.0 / n_steps
    for t in t_values:
        v = flow(z, t.repeat(z.size(0)))
        z = z + dt * v
    return z


In [6]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np

def train_epoch(train_loader, vae, flow, optimizer,
                lambda_kl=1e-2,
                temperature=0.07, queue_size=4098, device="cuda", epoch = 0):
    vae.train()
    flow.train()
    total_loss, total_fm, total_enc, total_kl = 0, 0, 0, 0
    criterion = QueueInfoNCELoss(dim=1536, temperature=temperature, queue_size=queue_size).to(device)

    for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch: {epoch}"):
        context = X_batch.to(device)
        image = y_batch.to(device)

        z0, mu, log_sigma, recon = vae(context)
        z1 = image

        t = torch.rand(z0.size(0), 1, device=device)
        z_t = (1 - t) * z0 + t * z1
        v_hat = z1 - z0
        v_pred = flow(z_t, t.squeeze())

        L_Enc = criterion(z0, z1)
        L_FM = F.mse_loss(v_pred, v_hat)
        L_KL = vae.kl_loss(mu, log_sigma)

        loss = L_FM + L_Enc + lambda_kl * L_KL

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_fm += L_FM.item()
        total_enc += L_Enc.item()
        total_kl += L_KL.item()

        with torch.no_grad():
          img_keys = F.normalize(image, dim=1).detach()
          # put them into the queue
          criterion._enqueue(keys=img_keys)

    n = len(train_loader)
    return {
        "loss": total_loss / n,
        "L_FM": total_fm / n,
        "L_Enc": total_enc / n,
        "L_KL": total_kl / n
    }


@torch.no_grad()
def validate_epoch(val_loader, vae, flow, device="cuda", n_steps=20):
    vae.eval()
    flow.eval()
    cos_sims = []

    for X_batch, y_batch in tqdm(val_loader, desc="Validation"):
        context = X_batch.to(device)
        image = y_batch.to(device)

        # Encode to z0
        z0, _, _, _ = vae(context)

        # Integrate flow to predict target embedding
        z1_pred = integrate_flow(flow, z0, n_steps=n_steps)
        z1_true = image

        # Compute cosine similarity between predicted and true image embeddings
        cos_sim = F.cosine_similarity(z1_pred, z1_true, dim=-1)
        cos_sims.append(cos_sim.cpu().numpy())

    cos_sims = np.concatenate(cos_sims)
    mean_cosine = np.mean(cos_sims)
    acc_80 = np.mean(cos_sims > 0.8)  # how often similarity > 0.8
    acc_90 = np.mean(cos_sims > 0.9)

    return {
        "mean_cosine": mean_cosine,
        "acc@0.8": acc_80,
        "acc@0.9": acc_90
    }


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class QueueInfoNCELoss(nn.Module):
    """
    One-directional (text → image) InfoNCE with optional sinusoidal time embedding.
    """
    def __init__(self, dim, temperature=0.07, queue_size=4096, use_time_embedding=True):
        super().__init__()
        self.temperature = temperature
        self.queue_size = queue_size

        # Single queue for image embeddings
        self.register_buffer("queue", F.normalize(torch.randn(queue_size, dim), dim=1))
        self.register_buffer("queue_ptr", torch.zeros(1, dtype=torch.long))



    # ------------------------------
    # Queue Management
    # ------------------------------
    @torch.no_grad()
    def _enqueue(self, keys):
        """Add image keys (B, dim) to the circular queue after backward()."""
        bsz = keys.shape[0]
        keys = F.normalize(keys, dim=1)

        ptr = int(self.queue_ptr.item())
        end_ptr = (ptr + bsz) % self.queue_size

        if end_ptr > ptr:
            self.queue[ptr:end_ptr] = keys
        else:
            first_len = self.queue_size - ptr
            self.queue[ptr:] = keys[:first_len]
            self.queue[:end_ptr] = keys[first_len:]

        self.queue_ptr[0] = end_ptr

    # ------------------------------
    # Forward Pass
    # ------------------------------
    def forward(self, z_text, z_img):
        """
        z_text: (B, dim) predicted text→image latent
        z_img:  (B, dim) target image latent
        """

        # Normalize embeddings
        z_text = F.normalize(z_text, dim=1)
        z_img = F.normalize(z_img, dim=1)

        # Positive logits: (B, 1)
        l_pos = torch.sum(z_text * z_img, dim=-1, keepdim=True)

        # Negatives: (B, queue_size)
        l_neg = torch.matmul(z_text, self.queue.T)

        # Combine and scale by temperature
        logits = torch.cat([l_pos, l_neg], dim=1) / self.temperature
        labels = torch.zeros(logits.size(0), dtype=torch.long, device=z_text.device)

        loss = F.cross_entropy(logits, labels)
        return loss


In [8]:
# ====== Procrustes initialization ======
def procrustes_init(text_embs, img_embs):
    """
    text_embs: (N, d_text)
    img_embs:  (N, d_img)
    returns: weight matrix (d_img, d_text)
    """
    # Center both
    X = text_embs - text_embs.mean(0, keepdim=True)
    Y = img_embs - img_embs.mean(0, keepdim=True)

    # Compute SVD of cross-covariance
    U, _, Vt = torch.linalg.svd(X.T @ Y, full_matrices=False)
    W = U @ Vt  # orthogonal map d_text→d_img
    return W.T   # shape (d_img, d_text) for nn.Linear weight


def apply_procrustes_init_to_final(model, text_sample, img_sample):
    """
    Apply Procrustes initialization to a model
    """
    with torch.no_grad():
        # Compute Procrustes matrix
        W = procrustes_init(text_embs=text_sample, img_embs=img_sample)

        # Apply to the appropriate layer
        applied = False
        for name, m in model.named_modules():
            if isinstance(m, nn.Linear):
                # Transformer: apply to first projection (proj_in)
                if isinstance(model, TransformerFlow) and name.endswith("output"):
                    print(m.weight.shape, W.shape)
                    if m.weight.shape == W.shape:
                        m.weight.copy_(W)
                        applied = True
                        break
        if not applied:
            print("⚠️ Warning: Could not find matching layer for Procrustes init")
    return model


### Load Data

In [13]:
# 3. Crossflow
# 4. Data Augmentation
# 5. Zero Shot Stitching
# 6. Diffusion Priors
# Configuration
EPOCHS = 60
BATCH_SIZE = 128
LR = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load data
train_data = load_data("drive/MyDrive/data/train/train.npz")
X, y, label = prepare_train_data(train_data)
DATASET_SIZE = len(X)
# Split train/val
# This is done only to measure generalization capabilities, you don't have to
# use a validation set (though we encourage this)
n_train = int(0.9 * len(X))
TRAIN_SPLIT = torch.zeros(len(X), dtype=torch.bool)
TRAIN_SPLIT[:n_train] = 1
X_train, X_val = X[TRAIN_SPLIT], X[~TRAIN_SPLIT]
y_train, y_val = y[TRAIN_SPLIT], y[~TRAIN_SPLIT]


train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
y_train.shape, X_train.shape, train_loader.batch_size, val_loader.batch_size

(125000,)
Train data: 125000 captions, 125000 images


(torch.Size([112500, 1536]), torch.Size([112500, 1024]), 128, 128)

In [14]:

vae = ContextMapperVAE(
    input_dim=1024, latent_dim=1536,
    num_layers=3, hidden_dim=1024, dropout=0.1
).to(DEVICE)

flow = TransformerFlow(
    latent_dim=1536, num_layers=6,
    num_heads=8, ff_dim=1024, dropout=0.1
).to(DEVICE)

optimizer = torch.optim.AdamW(
    list(vae.parameters()) + list(flow.parameters()),
    lr=1e-4, weight_decay=0.01
)

procrustes_init = False
if procrustes_init:
  print("Computing Procrustes initialization...")
  text_list, img_list = [], []
  for i, (X, y) in enumerate(train_loader):
      text_list.append(X.cpu())
      img_list.append(y.cpu())
      if sum(t.shape[0] for t in text_list) >= 20000:
            break
  text_sample = torch.cat(text_list, dim=0)[:20000]
  img_sample = torch.cat(img_list, dim=0)[:20000]
  flow = apply_procrustes_init_to_final(flow, text_sample, img_sample)

for epoch in range(EPOCHS):
    train_metrics = train_epoch(train_loader, vae, flow, optimizer, device=DEVICE, epoch=epoch)
    val_metrics = validate_epoch(val_loader, vae, flow, device=DEVICE)
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    print(f"Train: {train_metrics}")
    print(f"Val: {val_metrics}")

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(
Validation: 100%|██████████| 98/98 [00:09<00:00, 10.15it/s]



Epoch 1/60
Train: {'loss': 9.143248772865268, 'L_FM': 1.0541024583055976, 'L_Enc': 7.68200744893635, 'L_KL': 40.71389145791463}
Val: {'mean_cosine': np.float32(0.48039296), 'acc@0.8': np.float64(0.0), 'acc@0.9': np.float64(0.0)}


Validation: 100%|██████████| 98/98 [00:09<00:00, 10.11it/s]



Epoch 2/60
Train: {'loss': 8.60262702837739, 'L_FM': 0.7771246161341532, 'L_Enc': 7.287264568406974, 'L_KL': 53.82378841395807}
Val: {'mean_cosine': np.float32(0.53193843), 'acc@0.8': np.float64(0.0), 'acc@0.9': np.float64(0.0)}


Validation: 100%|██████████| 98/98 [00:09<00:00, 10.09it/s]


...


Epoch 45/60
Train: {'loss': 7.569884255748831, 'L_FM': 0.39846958391224296, 'L_Enc': 6.204688200229128, 'L_KL': 96.67265067246994}
Val: {'mean_cosine': np.float32(0.7243731), 'acc@0.8': np.float64(0.0976), 'acc@0.9': np.float64(0.0)}


Validation: 100%|██████████| 98/98 [00:09<00:00, 10.03it/s]



Epoch 46/60
Train: {'loss': 7.566512099711968, 'L_FM': 0.39779314849699454, 'L_Enc': 6.199773472730617, 'L_KL': 96.89455099615763}
Val: {'mean_cosine': np.float32(0.7244087), 'acc@0.8': np.float64(0.09752), 'acc@0.9': np.float64(0.00016)}


Epoch: 46:  53%|█████▎    | 463/879 [00:23<00:21, 19.79it/s]


KeyboardInterrupt: 

### Training and Hyperparameter Optimization

### Inference

In [17]:

test_data = load_data("drive/MyDrive/data/test/test.clean.npz")

test_embds = test_data['captions/embeddings']
test_embds = torch.from_numpy(test_embds).float().to(DEVICE)

with torch.no_grad():
    # Encode to z0
    z0, _, _, _ = vae(test_embds)
    # Integrate flow to predict target embedding
    z1_pred = integrate_flow(flow, z0, n_steps=50)

submission = generate_submission(test_data['captions/ids'], z1_pred, 'drive/MyDrive/data/crossflow_submission.csv')
MODEL_PATH = "drive/MyDrive/data//models/crossflow.pth"
print(f"Model saved to: {MODEL_PATH}")

Generating submission file...
✓ Saved submission to drive/MyDrive/data/crossflow_submission.csv
Model saved to: drive/MyDrive/data//models/crossflow.pth
